In [1]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME") 
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [2]:
from sqlalchemy import create_engine, text
import pymysql
import pandas as pd
db_connection_str = f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
db_connection = create_engine(db_connection_str)
conn = db_connection.connect()

## 시군구별 등록장애인수 전처리

In [4]:
sgis_disable = pd.read_sql_query(text("SELECT * FROM sgis_sigungu_disabled_cnt"), conn)
# '강원특별자치도' -> '강원도', '전북특별자치도' -> '전라북도'로 변경하여 새 컬럼 생성
sgis_disable['행정구역_수정'] = sgis_disable['행정구역'].str.replace('강원특별자치도', '강원도').str.replace('전북특별자치도', '전라북도')
sgis_disable.head()

,행정구역,순위,장애인수(명),비율(%),행정구역_수정
0,경기도 부천시,1.0,37505,1.42,경기도 부천시
1,경기도 남양주시,2.0,33172,1.26,경기도 남양주시
2,경기도 화성시,3.0,31921,1.21,경기도 화성시
3,서울특별시 강서구,4.0,28319,1.07,서울특별시 강서구
4,대구광역시 달서구,5.0,28314,1.07,대구광역시 달서구


In [93]:
sgis_disable = sgis_disable[['행정구역_수정', '장애인수(명)']]
sgis_disable

,행정구역_수정,장애인수(명)
0,경기도 부천시,37505
1,경기도 남양주시,33172
2,경기도 화성시,31921
3,서울특별시 강서구,28319
4,대구광역시 달서구,28314
...,...,...
245,강원도 화천군,1697
246,인천광역시 옹진군,1600
247,강원도 양구군,1438
248,경상북도 울릉군,535


## 시군구별 전체 인구수 전처리

In [ ]:
popul = pd.read_sql_query(text("SELECT * FROM sigungu_population"), conn)
# '강원특별자치도' -> '강원도', '전북특별자치도' -> '전라북도'로 변경하여 새 컬럼 생성
popul['행정구역_수정'] = popul['행정구역'].str.replace('강원특별자치도', '강원도').str.replace('전북특별자치도', '전라북도')

# 1. 합산할 3개 구의 행을 선택합니다.
target_rows = popul[popul['행정구역_수정'].isin(['경기도 부천시 원미구', '경기도 부천시 소사구', '경기도 부천시 오정구'])]

# 2. 숫자 데이터들을 모두 합산하여 새로운 행(Series)을 만듭니다.
new_row_series = target_rows.sum(numeric_only=True)

# 3. 새로운 행의 '행정구역_수정' 이름을 '경기도 부천시'로 지정합니다.
new_row_series['행정구역_수정'] = '경기도 부천시'

# 4. 기존 데이터프레임에 합산된 새로운 행을 추가합니다.
# Series를 DataFrame으로 변환한 후 concat을 사용합니다.
popul = pd.concat([popul, pd.DataFrame([new_row_series])], ignore_index=True)
popul.head()

,행정구역,순위,2023년 수(명),비율(%),행정구역_수정
0,경기도 화성시,1.0,974156.0,1.88,경기도 화성시
1,경기도 남양주시,2.0,716123.0,1.38,경기도 남양주시
2,서울특별시 송파구,3.0,630744.0,1.22,서울특별시 송파구
3,인천광역시 서구,4.0,621848.0,1.20,인천광역시 서구
4,경기도 평택시,5.0,605678.0,1.17,경기도 평택시


In [12]:
popul[popul['행정구역_수정'].str.contains('경기도 부천시')]

,행정구역,순위,2023년 수(명),비율(%),행정구역_수정
34,경기도 부천시 원미구,35.0,405077.0,0.78,경기도 부천시 원미구
94,경기도 부천시 소사구,95.0,242773.0,0.47,경기도 부천시 소사구
134,경기도 부천시 오정구,135.0,155662.0,0.30,경기도 부천시 오정구
252,NaN,265.0,803512.0,1.55,경기도 부천시


In [102]:
#--- 삭제할 행 목록 정의 ---
values_to_remove = ['경기도 부천시 원미구', '경기도 부천시 소사구', '경기도 부천시 오정구']

#--- 핵심 코드: 특정 값을 포함하는 행 삭제 ---
popul = popul[~popul['행정구역_수정'].isin(values_to_remove)]
popul = popul[['행정구역_수정', '2023년 수(명)']].rename(columns={'2023년 수(명)':'인구수'})
popul

,행정구역_수정,인구수
0,경기도 화성시,974156.0
1,경기도 남양주시,716123.0
2,서울특별시 송파구,630744.0
3,인천광역시 서구,621848.0
4,경기도 평택시,605678.0
...,...,...
248,전라북도 장수군,19914.0
249,인천광역시 옹진군,19286.0
250,경상북도 영양군,15197.0
251,경상북도 울릉군,8418.0


## 시군구별 저소득층(기초생활수급자) 수 전처리

In [ ]:
gisangsu = pd.read_sql_query(text("SELECT * FROM sigungu_gisangsu"), conn)
gisangsu.head()

,사업명,기준년월,시도,시군구,수급권자수,수급가구수
0,기초생활보장(맞춤형급여),2023-12,서울특별시,종로구,5797,4996
1,기초생활보장(맞춤형급여),2023-12,서울특별시,중구,6263,5175
2,기초생활보장(맞춤형급여),2023-12,서울특별시,용산구,9290,7346
3,기초생활보장(맞춤형급여),2023-12,서울특별시,성동구,10561,8154
4,기초생활보장(맞춤형급여),2023-12,서울특별시,광진구,14078,10553


In [23]:
gisangsu = gisangsu[gisangsu['사업명'] == '기초생활보장(맞춤형급여)'].drop(columns=['기준년월'])
gisangsu.head()

,사업명,시도,시군구,수급권자수,수급가구수
0,기초생활보장(맞춤형급여),서울특별시,종로구,5797,4996
1,기초생활보장(맞춤형급여),서울특별시,중구,6263,5175
2,기초생활보장(맞춤형급여),서울특별시,용산구,9290,7346
3,기초생활보장(맞춤형급여),서울특별시,성동구,10561,8154
4,기초생활보장(맞춤형급여),서울특별시,광진구,14078,10553


In [24]:
gisangsu.시군구.unique()

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '서구', '동구',
       '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '연제구',
       '수영구', '사상구', '기장군', '수성구', '달서구', '달성군', '군위군', '미추홀구', '연수구',
       '남동구', '부평구', '계양구', '강화군', '옹진군', '광산구', '유성구', '대덕구', '울주군',
       '세종특별자치시', '수원시', '수원시장안구', '수원시권선구', '수원시팔달구', '수원시영통구', '성남시',
       '성남시수정구', '성남시중원구', '성남시분당구', '의정부시', '안양시', '안양시만안구', '안양시동안구',
       '부천시', '광명시', '평택시', '동두천시', '안산시', '안산시상록구', '안산시단원구', '고양시',
       '고양시덕양구', '고양시일산동구', '고양시일산서구', '과천시', '구리시', '남양주시', '오산시', '시흥시',
       '군포시', '의왕시', '하남시', '용인시', '용인시처인구', '용인시기흥구', '용인시수지구', '파주시',
       '이천시', '안성시', '김포시', '화성시', '광주시', '양주시', '포천시', '여주시', '연천군',
       '가평군', '양평군', '청주시', '청주시상당구', '청주시서원구', '청주시흥덕구', '청주시청원구', '충주시',
       '제천시', '보은군', '옥천군', '영동군', '증평군', '진천군', '괴산군', '음성군', '단양군',
   

In [25]:
# 1. 세종특별자치시의 '소계'를 '세종시'로 변경
# .loc를 사용하여 특정 조건의 행과 열을 지정하여 값을 변경
gisangsu.loc[gisangsu['시도'] == '세종특별자치시', '시군구'] = '세종시'

# 2. '수원시권선구' 형태의 문자열을 분리하는 함수 정의
def split_city_district(name):
    # '시'의 위치를 찾음
    si_index = name.find('시')
    # '시'가 있고, 마지막 글자가 아니며, 바로 뒤에 공백이 없는 경우
    if si_index > 0 and si_index < len(name) - 1 and name[si_index+1] != ' ':
        return name[:si_index+1] + ' ' + name[si_index+1:]
    return name # 그 외의 경우는 원래 이름 반환

# 3. 위에서 정의한 함수를 '시군구' 컬럼에 적용
gisangsu['시군구'] = gisangsu['시군구'].apply(split_city_district)

# 4. '시도' 컬럼의 명칭 변경
rename_dict = {
    '강원특별자치도': '강원도',
    '전북특별자치도': '전라북도'
}
gisangsu['시도'] = gisangsu['시도'].replace(rename_dict)

# 5. 두 컬럼을 합쳐 '행정구역' 파생변수 생성
gisangsu['행정구역'] = gisangsu['시도'] + ' ' + gisangsu['시군구']

# 6. 불필요한 원본 컬럼들 제거
gisangsu_final = gisangsu.drop(columns=['시도', '시군구'])

print("\n--- 최종 결과 데이터프레임 ---")
print(gisangsu_final)


--- 최종 결과 데이터프레임 ---
               사업명  수급권자수  수급가구수       행정구역
0    기초생활보장(맞춤형급여)   5797   4996  서울특별시 종로구
1    기초생활보장(맞춤형급여)   6263   5175   서울특별시 중구
2    기초생활보장(맞춤형급여)   9290   7346  서울특별시 용산구
3    기초생활보장(맞춤형급여)  10561   8154  서울특별시 성동구
4    기초생활보장(맞춤형급여)  14078  10553  서울특별시 광진구
..             ...    ...    ...        ...
256  기초생활보장(맞춤형급여)   1218    976    강원도 화천군
257  기초생활보장(맞춤형급여)   1136    884    강원도 양구군
258  기초생활보장(맞춤형급여)   1368   1076    강원도 인제군
259  기초생활보장(맞춤형급여)   1519   1219    강원도 고성군
260  기초생활보장(맞춤형급여)   1767   1450    강원도 양양군

[261 rows x 4 columns]


In [110]:
gisangsu = gisangsu_final[['행정구역', '수급권자수']].rename(columns={'행정구역':'행정구역_수정'})

In [113]:
tmp = pd.merge(gisangsu, popul, on='행정구역_수정', how='left')
tmp[tmp['인구수'].isnull()]['행정구역_수정']

76      경기도 수원시
81      경기도 성남시
86      경기도 안양시
93      경기도 안산시
96      경기도 고양시
108     경기도 용인시
124    충청북도 청주시
139    충청남도 천안시
156    전라북도 전주시
194    경상북도 포항시
218    경상남도 창원시
Name: 행정구역_수정, dtype: object

In [114]:
#--- 핵심 코드: tmp의 '행정구역_수정' 값을 gisangsu에서 제외 ---

# 1. 제외할 행정구역 목록을 만듭니다.
exclude_list = tmp[tmp['인구수'].isnull()]['행정구역_수정']

# 2. gisangsu의 '행정구역_수정' 값이 exclude_list에 포함되지 않는(~isin) 행만 선택합니다.
gisangsu = gisangsu[~gisangsu['행정구역_수정'].isin(exclude_list)]
gisangsu

,행정구역_수정,수급권자수
0,서울특별시 종로구,5797
1,서울특별시 중구,6263
2,서울특별시 용산구,9290
3,서울특별시 성동구,10561
4,서울특별시 광진구,14078
...,...,...
256,강원도 화천군,1218
257,강원도 양구군,1136
258,강원도 인제군,1368
259,강원도 고성군,1519


## 결혼 이민자 데이터 전처리

In [ ]:
migrate = pd.read_sql_query(text('SELECT * FROM sigungu_migrate'), conn)
migrate.head()

,행정구역(시군구)별(1),행정구역(시군구)별(2),총계
0,강원특별자치도,소계,3989
1,강원특별자치도,강릉시,481
2,강원특별자치도,고성군,71
3,강원특별자치도,동해시,230
4,강원특별자치도,삼척시,161


In [31]:
migrate['행정구역(시군구)별(2)'].unique()

array(['소계', '강릉시', '고성군', '동해시', '삼척시', '속초시', '양구군', '양양군', '영월군',
       '원주시', '인제군', '정선군', '철원군', '춘천시', '태백시', '평창군', '홍천군', '화천군',
       '횡성군', '가평군', '고양시 덕양구', '고양시 일산동구', '고양시 일산서구', '과천시', '광명시',
       '광주시', '구리시', '군포시', '김포시', '남양주시', '동두천시', '부천시', '성남시 분당구',
       '성남시 수정구', '성남시 중원구', '수원시 권선구', '수원시 영통구', '수원시 장안구', '수원시 팔달구',
       '시흥시', '안산시 단원구', '안산시 상록구', '안성시', '안양시 동안구', '안양시 만안구', '양주시',
       '양평군', '여주시', '연천군', '오산시', '용인시 기흥구', '용인시 수지구', '용인시 처인구', '의왕시',
       '의정부시', '이천시', '파주시', '평택시', '포천시', '하남시', '화성시', '거제시', '거창군',
       '김해시', '남해군', '밀양시', '사천시', '산청군', '양산시', '의령군', '진주시', '창녕군',
       '창원시 마산합포구', '창원시 마산회원구', '창원시 성산구', '창원시 의창구', '창원시 진해구', '통영시',
       '하동군', '함안군', '함양군', '합천군', '경산시', '경주시', '고령군', '구미시', '김천시',
       '문경시', '봉화군', '상주시', '성주군', '안동시', '영덕군', '영양군', '영주시', '영천시',
       '예천군', '울릉군', '울진군', '의성군', '청도군', '청송군', '칠곡군', '포항시 남구',
       '포항시 북구', '광산구', '남구', '동구', '북구', '서구', '군위군', '달서구', '달성군',
       '수성구',

In [118]:
import pandas as pd

# 2. '소계' 행 제거 (세종특별자치시는 예외로 남김)
# 조건1: '소계'가 아니거나 (|) 조건2: '세종특별자치시'인 경우만 남김
condition = (migrate['행정구역(시군구)별(2)'] != '소계') | (migrate['행정구역(시군구)별(1)'] == '세종특별자치시')
migrate_filtered = migrate[condition].copy()

# 3. 세종특별자치시의 '소계'를 '세종시'로 변경
# .loc를 사용하여 특정 조건의 행과 열을 지정하여 값을 변경
migrate_filtered.loc[migrate_filtered['행정구역(시군구)별(1)'] == '세종특별자치시', '행정구역(시군구)별(2)'] = '세종시'

# 4. '행정구역(시군구)별(1)' 컬럼의 명칭 변경
rename_dict = {
    '강원특별자치도': '강원도',
    '전북특별자치도': '전라북도'
}
migrate_filtered['행정구역(시군구)별(1)'] = migrate_filtered['행정구역(시군구)별(1)'].replace(rename_dict)

# 5. 두 컬럼을 합쳐 '행정구역' 파생변수 생성
migrate_filtered['행정구역'] = migrate_filtered['행정구역(시군구)별(1)'] + ' ' + migrate_filtered['행정구역(시군구)별(2)']

# 6. 불필요한 원본 컬럼들 제거
migrate_final = migrate_filtered.drop(columns=['행정구역(시군구)별(1)', '행정구역(시군구)별(2)'])

migrate_final = migrate_final[['행정구역', '총계']].rename(columns={'행정구역':'행정구역_수정', '총계' : '결혼이민자'})

print("\n--- 최종 결과 데이터프레임 ---")
print(migrate_final)


--- 최종 결과 데이터프레임 ---
          행정구역_수정  결혼이민자
1         강원도 강릉시    481
2         강원도 고성군     71
3         강원도 동해시    230
4         강원도 삼척시    161
5         강원도 속초시    218
..            ...    ...
261  충청북도 청주시 상당구    488
262  충청북도 청주시 서원구    448
263  충청북도 청주시 청원구    602
264  충청북도 청주시 흥덕구    831
265      충청북도 충주시    712

[250 rows x 2 columns]


In [119]:
migrate = migrate_final.copy()

## 결혼이민자 데이터, 기초생활수급자 데이터 병합

In [120]:
migrate_gisangsu = pd.merge(migrate, gisangsu, on='행정구역_수정')
migrate_gisangsu[migrate_gisangsu['결혼이민자'].isnull()]

,행정구역_수정,결혼이민자,수급권자수


In [122]:
migrate_gisangsu[migrate_gisangsu['행정구역_수정'].str.contains('경기도')].head()

,행정구역_수정,결혼이민자,수급권자수
18,경기도 가평군,461,4420
19,경기도 고양시 덕양구,1418,21024
20,경기도 고양시 일산동구,887,8943
21,경기도 고양시 일산서구,716,6143
22,경기도 과천시,100,1098


In [124]:
migrate_gisangsu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   행정구역_수정  250 non-null    object
 1   결혼이민자    250 non-null    int64 
 2   수급권자수    250 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.0+ KB


## 시군구별 고령인구 수 전처리

In [ ]:
oldman = pd.read_sql_query(text("SELECT * FROM sigungu_oldman"), conn)
# '강원특별자치도' -> '강원도', '전북특별자치도' -> '전라북도'로 변경하여 새 컬럼 생성
oldman['행정구역_수정'] = oldman['행정구역'].str.replace('강원특별자치도', '강원도').str.replace('전북특별자치도', '전라북도')

# 1. 합산할 3개 구의 행을 선택합니다.
target_rows = oldman[oldman['행정구역_수정'].isin(['경기도 부천시 원미구', '경기도 부천시 소사구', '경기도 부천시 오정구'])]

# 2. 숫자 데이터들을 모두 합산하여 새로운 행(Series)을 만듭니다.
new_row_series = target_rows.sum(numeric_only=True)

# 3. 새로운 행의 '행정구역_수정' 이름을 '경기도 부천시'로 지정합니다.
new_row_series['행정구역_수정'] = '경기도 부천시'

# 4. 기존 데이터프레임에 합산된 새로운 행을 추가합니다.
# Series를 DataFrame으로 변환한 후 concat을 사용합니다.
oldman = pd.concat([oldman, pd.DataFrame([new_row_series])], ignore_index=True)
oldman.head()

,행정구역,순위,2023년 수(명),비율(%),행정구역_수정
0,경기도 남양주시,1.0,120439.0,1.25,경기도 남양주시
1,서울특별시 송파구,2.0,105982.0,1.10,서울특별시 송파구
2,서울특별시 강서구,3.0,100013.0,1.04,서울특별시 강서구
3,경기도 화성시,4.0,95428.0,0.99,경기도 화성시
4,서울특별시 노원구,5.0,94010.0,0.98,서울특별시 노원구


In [127]:
#--- 삭제할 행 목록 정의 ---
values_to_remove = ['경기도 부천시 원미구', '경기도 부천시 소사구', '경기도 부천시 오정구']

#--- 핵심 코드: 특정 값을 포함하는 행 삭제 ---
oldman = oldman[~oldman['행정구역_수정'].isin(values_to_remove)]
oldman = oldman[['행정구역_수정', '2023년 수(명)']].rename(columns={'2023년 수(명)':'고령인구수'})
oldman

,행정구역_수정,고령인구수
0,경기도 남양주시,120439.0
1,서울특별시 송파구,105982.0
2,서울특별시 강서구,100013.0
3,경기도 화성시,95428.0
4,서울특별시 노원구,94010.0
...,...,...
248,인천광역시 옹진군,5940.0
249,강원도 화천군,5913.0
250,강원도 양구군,5118.0
251,경상북도 울릉군,2361.0


## 시군구별 은행 점포수 전처리

In [ ]:
bank_addr = pd.read_sql_query(text("SELECT * FROM bank_address_cleaned"), conn)
bank_addr.head()

,은행,지점,주소,시도,코드,시군구,SIGUNGU_CD,행정구역_수정
0,BNK경남은행,LH,경상남도 진주시 충무공동 충의로 19 LH진주사옥,경상남도,38,진주시,38030.0,경상남도 진주시
1,BNK경남은행,가산디지털,서울특별시 금천구 가산동 가산디지털 1로,서울특별시,11,금천구,11180.0,서울특별시 금천구
2,BNK경남은행,가음정금융센터,경상남도 창원시 성산구 대정로 36,경상남도,38,창원시 성산구,38112.0,경상남도 창원시 성산구
3,BNK경남은행,강남,서울특별시 강남구 대치동 영동대로 421,서울특별시,11,강남구,11230.0,서울특별시 강남구
4,BNK경남은행,거제고현,경상남도 거제시 고현동 거제중앙로 1895,경상남도,38,거제시,38090.0,경상남도 거제시


In [50]:
# 시군구별 점포 수 집계
branch_count = bank_addr.groupby('행정구역_수정')[["은행"]].count().reset_index().rename(columns = {"은행" : "점포수"})
branch_count

,행정구역_수정,점포수
0,강원도 강릉시,16
1,강원도 고성군,2
2,강원도 동해시,8
3,강원도 삼척시,5
4,강원도 속초시,8
...,...,...
244,충청북도 청주시 상당구,16
245,충청북도 청주시 서원구,12
246,충청북도 청주시 청원구,18
247,충청북도 청주시 흥덕구,27


In [130]:
x = pd.merge(branch_count, popul, how='outer')
x[x.점포수.isnull()]

,행정구역_수정,점포수,인구수
179,인천광역시 옹진군,NaN,19286.0


=> 옹진군에는 점포가 없음

## 버스정류장 수, 지하철역 수 데이터 전처리 (QGIS로 전처리함, EPSG:5179 좌표계로 통일)

In [ ]:
bus = pd.read_sql_query(text('SELECT * FROM bus_stop'), conn)
bus

,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명
0,ADB354000001,길안정류장,36.458658,128.891228,2024-10-28,540001.0,37040,경상북도 안동시,안동BIS
1,ADB354000002,고란.계명산휴양림입구,36.397708,128.924029,2024-10-28,540002.0,37040,경상북도 안동시,안동BIS
2,ADB354000003,금곡리,36.397504,128.926821,2024-10-28,540003.0,37040,경상북도 안동시,안동BIS
3,ADB354000004,금소리1,36.509449,128.849671,2024-10-28,540004.0,37040,경상북도 안동시,안동BIS
4,ADB354000005,금소리5,36.503629,128.854783,2024-10-28,540005.0,37040,경상북도 안동시,안동BIS
...,...,...,...,...,...,...,...,...,...
206017,YEB328012050,중마시장,34.936978,127.697847,2024-10-28,0.0,36020,전라남도 여수시,여수BIS
206018,YEB328013028,사곡,34.961730,127.628080,2024-10-28,3283028.0,36020,전라남도 여수시,여수BIS
206019,YEB328013038,북부정류장,34.979260,127.582940,2024-10-28,3283038.0,36020,전라남도 여수시,여수BIS
206020,YEB328013041,매화아파트,34.983380,127.580570,2024-10-28,3283041.0,36020,전라남도 여수시,여수BIS


=> 위치에 따라 속성 결합 수행, 해당 폴리곤(시군구) 안에 포함된 포인트(버스정류장) 수를 집계하여 csv생성

In [ ]:
subway = pd.read_sql_query(text('SELECT * FROM subway'), conn)
subway.head()

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
0,S112,4.19민주묘지,L11UI,우이신설선,April 19th National Cemetery,四一九民主墓地,일반역,NaN,NaN,37.649540,127.013710,우이신설경전철운영㈜,서울특별시 강북구 삼양로 519,-,2022-05-25 00:00:00
1,1907,가능역,I4102,경원선,Ganeung,佳 陵,일반역,NaN,NaN,37.748362,127.044251,한국철도공사,경기도 의정부시 평화로 633,1588-7788,2025-04-08 00:00:00
2,0350,가락시장,I1103,3호선,Garak Market,可樂市長,환승역,S1108,수도권 도시철도 8호선,37.492368,127.118101,서울교통공사,서울특별시 송파구 송파대로 지하257(가락동),02-6110-3501,2024-12-31 00:00:00
3,0817,가락시장,S1108,8호선,Garak Market,可樂市長,환승역,I1103,수도권 광역철도 3호선,37.493004,127.118279,서울교통공사,서울특별시 송파구 송파대로 지하257(가락동),02-6311-8171,2024-12-31 00:00:00
4,0746,가산디지털단지,S1107,7호선,Gasan Digital Complex,加山디지털團地,환승역,I4101,수도권 광역철도 1호선,37.480376,126.882704,서울교통공사,서울특별시 금천구 벚꽃로 309(가산동),02-6311-7461,2024-12-31 00:00:00


=> QGIS로 시각화했을 때, **대경선** 노선의 역들과 **현충로, 양원역**의 위경도 위치가 이상함을 인지하고 수기로 다시 재할당

In [66]:
# stations_to_exclude = ['현충로', '양원역']

# remainder = subway[(subway['노선명']=='대경선') | (subway['역사명'].isin(stations_to_exclude))]

# remainder

,역번호,역사명,노선번호,노선명,영문역사명,한자역사명,환승역구분,환승노선번호,환승노선명,역위도,역경도,운영기관명,역사도로명주소,역사전화번호,데이터기준일자
63,7008,경산역,I27K7,대경선,Gyeongsan,慶 山,일반역,NaN,NaN,35.819200,128.727600,한국철도공사,경북 경산시 중앙로1,1588-7788,2025-04-08 00:00:00
126,7001,구미역,I27K7,대경선,Gumi,龜 尾,일반역,NaN,NaN,36.128200,128.331000,한국철도공사,경북 구미시 구미중앙로 76,1588-7788,2025-04-08 00:00:00
240,7006,대구역,I27K7,대경선,Daegu,大 邱,환승역,S2701,대구도시철도 1호선,35.876000,128.596100,한국철도공사,대구 북구 태평로 161,1588-7788,2025-04-08 00:00:00
291,7007,동대구역,I27K7,대경선,Dongdaegu,東大邱,환승역,S2701,대구도시철도 1호선,35.679100,128.627000,한국철도공사,대구 동구 동대구로 550,1588-7788,2025-04-08 00:00:00
482,7002,사곡역,I27K7,대경선,Sagok,沙 谷,일반역,NaN,NaN,35.098800,128.356600,한국철도공사,경북 구미시 상사서로 173-16,1588-7788,2025-04-08 00:00:00
529,7005,서대구역,I27K7,대경선,Seodaegu,西大邱,일반역,NaN,NaN,35.881400,128.542200,한국철도공사,대구 서구 와룡로 527,1588-7788,2025-04-08 00:00:00
723,1204,양원역,I4108,경의중앙선,Yangwon,養 源,일반역,NaN,NaN,36.963729,129.091321,한국철도공사,서울시 중랑구 송림길 147(망우동),1588-7788,2025-04-08 00:00:00
795,7004,왜관역,I27K7,대경선,Waegwan,倭 館,일반역,NaN,NaN,35.992100,128.400100,한국철도공사,경북 칠곡군 왜관읍 중앙로8길 10,1588-7788,2025-04-08 00:00:00
1059,0126,현충로,S2701,대구 도시철도 1호선,Hyeonchungno,顯忠路,일반역,NaN,NaN,35.502720,128.345300,대구교통공사,남구 대명로 지하222 (대명동),053-651-7756,2025-03-31 00:00:00


In [67]:
# # 2. 업데이트할 새로운 위경도 데이터로 데이터프레임 생성
# update_data = {
#     '역사명': ['경산역', '구미역', '대구역', '동대구역', '사곡역', '서대구역', '양원역', '왜관역', '현충로'],
#     '역위도': [35.819389017, 36.128330116, 35.875884949, 35.879436144, 36.098718204, 35.881479777, 37.606554361, 35.991893070, 35.841174924],
#     '역경도': [128.727508397, 128.330939456, 128.596129208, 128.628775532, 128.356651456, 128.540175582, 127.107921489, 128.400011399, 128.581924238]
# }
# update_df = pd.DataFrame(update_data)
# print("\n--- 2. 업데이트용 데이터프레임 ---")
# print(update_df)


# # 3. '역사명'을 인덱스로 설정하여 업데이트 기준을 맞춤
# subway.set_index('역사명', inplace=True)
# update_df.set_index('역사명', inplace=True)

# # 4. update 메서드로 값 덮어쓰기
# # df의 인덱스와 일치하는 update_df의 인덱스를 찾아 값을 업데이트합니다.
# subway.update(update_df)

# # 5. 인덱스를 다시 일반 컬럼으로 복원
# subway.reset_index(inplace=True)

# print("\n--- 3. 최종 업데이트 후 데이터프레임 ---")
# print(subway)


--- 2. 업데이트용 데이터프레임 ---
    역사명        역위도         역경도
0   경산역  35.819389  128.727508
1   구미역  36.128330  128.330939
2   대구역  35.875885  128.596129
3  동대구역  35.879436  128.628776
4   사곡역  36.098718  128.356651
5  서대구역  35.881480  128.540176
6   양원역  37.606554  127.107921
7   왜관역  35.991893  128.400011
8   현충로  35.841175  128.581924

--- 3. 최종 업데이트 후 데이터프레임 ---
            역사명   역번호   노선번호          노선명                         영문역사명  \
0      4.19민주묘지  S112  L11UI        우이신설선  April 19th National Cemetery   
1           가능역  1907  I4102          경원선                       Ganeung   
2          가락시장  0350  I1103          3호선                  Garak Market   
3          가락시장  0817  S1108          8호선                  Garak Market   
4       가산디지털단지  0746  S1107          7호선         Gasan Digital Complex   
...         ...   ...    ...          ...                           ...   
1082         효자  U121  L41U1          의정부                         Hyoja   
1083      효창공원앞  0627  S1106        

In [69]:
# subway.to_csv('data/subway.csv', index=False)

In [140]:
# bus_subway = pd.read_csv('data/시군구별_버스+지하철역_개수.csv')
bus_subway = bus_subway.fillna(0)
bus_subway

,BASE_DATE,SIGUNGU_CD,SIGUNGU_NM,정류장명_count,역번호_count
0,20231231,11010,종로구,85,14.0
1,20231231,11020,중구,93,24.0
2,20231231,11030,용산구,95,19.0
3,20231231,11040,성동구,44,20.0
4,20231231,11050,광진구,116,11.0
...,...,...,...,...,...
245,20231231,38580,함양군,652,0.0
246,20231231,38590,거창군,721,0.0
247,20231231,38600,합천군,816,0.0
248,20231231,39010,제주시,2376,0.0


In [141]:
bus_subway = bus_subway.iloc[:, 1:].rename(columns={'정류장명_count': '정류장 수', '역번호_count' :'지하철역 수'}).astype({'지하철역 수': 'int'})

### 시군구 교통인프라 확인을 위한 면적당 대중교통 정류장 수 데이터 생성

In [142]:
bus_subway['버정+지하철역'] = bus_subway['정류장 수'] + bus_subway['지하철역 수']
bus_subway.drop(['정류장 수', '지하철역 수'], axis=1, inplace=True)
bus_subway

,SIGUNGU_CD,SIGUNGU_NM,버정+지하철역
0,11010,종로구,99
1,11020,중구,117
2,11030,용산구,114
3,11040,성동구,64
4,11050,광진구,127
...,...,...,...
245,38580,함양군,652
246,38590,거창군,721
247,38600,합천군,816
248,39010,제주시,2376


In [143]:
area = pd.read_csv('data/시군구별_면적데이터.csv')
area

,BASE_DATE,SIGUNGU_CD,SIGUNGU_NM,area
0,20231231,11010,종로구,23990639
1,20231231,11020,중구,9990180
2,20231231,11030,용산구,21892568
3,20231231,11040,성동구,16817147
4,20231231,11050,광진구,17049005
...,...,...,...,...
245,20231231,38580,함양군,721003386
246,20231231,38590,거창군,799504067
247,20231231,38600,합천군,983776435
248,20231231,39010,제주시,984408987


In [145]:
bus_subway_area = pd.merge(bus_subway, area, on='SIGUNGU_CD')
bus_subway_area.head()

,SIGUNGU_CD,SIGUNGU_NM_x,버정+지하철역,BASE_DATE,SIGUNGU_NM_y,area
0,11010,종로구,99,20231231,종로구,23990639
1,11020,중구,117,20231231,중구,9990180
2,11030,용산구,114,20231231,용산구,21892568
3,11040,성동구,64,20231231,성동구,16817147
4,11050,광진구,127,20231231,광진구,17049005


In [147]:
bus_subway_area['시도코드'] = bus_subway_area['SIGUNGU_CD'].astype(str).str[:2].astype(int)
bus_subway_area.head()

,SIGUNGU_CD,SIGUNGU_NM_x,버정+지하철역,BASE_DATE,SIGUNGU_NM_y,area,시도코드
0,11010,종로구,99,20231231,종로구,23990639,11
1,11020,중구,117,20231231,중구,9990180,11
2,11030,용산구,114,20231231,용산구,21892568,11
3,11040,성동구,64,20231231,성동구,16817147,11
4,11050,광진구,127,20231231,광진구,17049005,11


In [148]:
sido = pd.read_csv('data/시도코드.csv', encoding = 'cp949', header = None, names = ['시도', '코드'])
sido.head()

,시도,코드
0,서울특별시,11
1,부산광역시,21
2,대구광역시,22
3,인천광역시,23
4,광주광역시,24


In [149]:
bus_subway_area = pd.merge(bus_subway_area, sido, left_on='시도코드', right_on='코드')
bus_subway_area.head()

,SIGUNGU_CD,SIGUNGU_NM_x,버정+지하철역,BASE_DATE,SIGUNGU_NM_y,area,시도코드,시도,코드
0,11010,종로구,99,20231231,종로구,23990639,11,서울특별시,11
1,11020,중구,117,20231231,중구,9990180,11,서울특별시,11
2,11030,용산구,114,20231231,용산구,21892568,11,서울특별시,11
3,11040,성동구,64,20231231,성동구,16817147,11,서울특별시,11
4,11050,광진구,127,20231231,광진구,17049005,11,서울특별시,11


In [152]:
bus_subway_area['행정구역_수정'] = bus_subway_area['시도'] + ' ' + bus_subway_area['SIGUNGU_NM_x']
bus_subway_area = bus_subway_area[['행정구역_수정', '버정+지하철역', 'area']]
bus_subway_area

,행정구역_수정,버정+지하철역,area
0,서울특별시 종로구,99,23990639
1,서울특별시 중구,117,9990180
2,서울특별시 용산구,114,21892568
3,서울특별시 성동구,64,16817147
4,서울특별시 광진구,127,17049005
...,...,...,...
245,경상남도 함양군,652,721003386
246,경상남도 거창군,721,799504067
247,경상남도 합천군,816,983776435
248,제주특별자치도 제주시,2376,984408987


In [156]:
bus_subway_area['trafic_infra'] = bus_subway_area['버정+지하철역'] / bus_subway_area['area']
bus_subway_area = bus_subway_area[['행정구역_수정','trafic_infra']]
bus_subway_area

C:\Users\ghtjq\AppData\Local\Temp\ipykernel_28220\537191000.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus_subway_area['trafic_infra'] = bus_subway_area['버정+지하철역'] / bus_subway_area['area']


,행정구역_수정,trafic_infra
0,서울특별시 종로구,4.126610e-06
1,서울특별시 중구,1.171150e-05
2,서울특별시 용산구,5.207247e-06
3,서울특별시 성동구,3.805640e-06
4,서울특별시 광진구,7.449115e-06
...,...,...
245,경상남도 함양군,9.042953e-07
246,경상남도 거창군,9.018090e-07
247,경상남도 합천군,8.294567e-07
248,제주특별자치도 제주시,2.413631e-06


In [158]:
bus_subway_area.to_sql(name='sigungu_trafic_infra', con=db_connection, if_exists='append',index=False)

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '221.155.195.6' (timed out)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

## 군집화를 위한 모든 데이터 병합

In [202]:
from functools import reduce

# 1. 병합할 데이터프레임들을 리스트에 담습니다.
data_frames = [sgis_disable, migrate_gisangsu, oldman, bus_subway_area, branch_count, popul]

# 2. reduce 함수를 사용하여 '행정구역_수정'을 기준으로 outer 병합을 실행합니다.
merged_df = reduce(lambda left, right: pd.merge(left, right, on='행정구역_수정', how='outer'), data_frames)

merged_df

,행정구역_수정,장애인수(명),결혼이민자,수급권자수,고령인구수,trafic_infra,점포수,인구수
0,강원도 강릉시,13387,481,13057,50763.0,2.305463e-08,16.0,213786.0
1,강원도 고성군,2231,71,1519,8408.0,1.511527e-09,2.0,27811.0
2,강원도 동해시,6657,230,5687,20568.0,3.860876e-08,8.0,86222.0
3,강원도 삼척시,5338,161,4084,17407.0,1.110651e-07,5.0,63542.0
4,강원도 속초시,4880,218,5661,18219.0,5.004629e-07,8.0,80497.0
...,...,...,...,...,...,...,...,...
245,충청북도 청주시 상당구,10693,488,9908,34704.0,2.164370e-06,16.0,197451.0
246,충청북도 청주시 서원구,9975,448,10275,34307.0,5.354256e-06,12.0,195222.0
247,충청북도 청주시 청원구,8896,602,6977,28970.0,3.738288e-06,18.0,196291.0
248,충청북도 청주시 흥덕구,10984,831,8857,36172.0,6.464511e-06,27.0,281394.0


In [203]:
merged_df.isnull().sum() # 옹진군에 은행 점포 없음

행정구역_수정         0
장애인수(명)         0
결혼이민자           0
수급권자수           0
고령인구수           0
trafic_infra    0
점포수             1
인구수             0
dtype: int64

In [204]:
merged_df = merged_df.fillna(1)
merged_df = merged_df.astype({'고령인구수': 'int', '점포수': 'int', '인구수': 'int'})

In [205]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   행정구역_수정       250 non-null    object 
 1   장애인수(명)       250 non-null    int64  
 2   결혼이민자         250 non-null    int64  
 3   수급권자수         250 non-null    int64  
 4   고령인구수         250 non-null    int32  
 5   trafic_infra  250 non-null    float64
 6   점포수           250 non-null    int32  
 7   인구수           250 non-null    int32  
dtypes: float64(1), int32(3), int64(3), object(1)
memory usage: 12.8+ KB


| 피처 (Feature) | 핵심 기준 (Core Criterion) | 최종 가중치 | 선정 이유 (Reasoning) |
|---|---|:---:|---|
| 점포 당 인구 수 | 서비스 공급 부족 | **40** | 가장 직접적인 급응 인프라 부족 지표 |
| 고령인구 비율 | 디지털/오프라인 의존도 | **25** | 오프라인 의존도가 가장 높은 집단 |
| 기초생활수급자 비율 | 사회/경제적 취약성 | **20** | 경제적 취약성 및 대면 상담 필요성 |
| 장애인구 비율 | 물리적 접근성 | **7** | 물리적 이동 제약의 직접적 지표 |
| 결혼이민자 비율 | 디지털/오프라인 의존도 | **5** | 언어/문화적 장벽으로 인한 대면 서비스 필요 |
| 면적 | 물리적 접근성 | **3** | 이동 거리에 대한 간접 지표 (보조 역할) |
| **총합** | | **100** | |

In [215]:
merged_df[merged_df['점포 당 인구수'] == min(merged_df['점포 당 인구수'])] # 점포 수를 1이라 가정했을 때, 점포당 인구수는 옹진군 == 19286

,행정구역_수정,장애인수(명),결혼이민자,수급권자수,고령인구수,trafic_infra,점포수,인구수,결혼이민자 비율,장애인구 비율,고령인구 비율,기초생활수급자 비율,점포 당 인구수
163,서울특별시 중구,5461,580,6263,24642,0.000012,129,127576,0.004546,0.042806,0.193155,0.049092,988.96124


In [216]:
# 1. 비율 관련 컬럼 4개 생성
merged_df['결혼이민자 비율'] = merged_df['결혼이민자'] / merged_df['인구수']
merged_df['장애인구 비율'] = merged_df['장애인수(명)'] / merged_df['인구수']
merged_df['고령인구 비율'] = merged_df['고령인구수'] / merged_df['인구수']
merged_df['기초생활수급자 비율'] = merged_df['수급권자수'] / merged_df['인구수']

# 2. 점포 당 인구수 컬럼 생성
merged_df['점포 당 인구수'] = merged_df['인구수'] / merged_df['점포수']

# 3. 교통인프라지수가 높을 수록 금융에 취약하지 않기때문에 다른 feature들과 비례하도록 -1 곱함
merged_df['trafic_infra'] = -1 * merged_df['trafic_infra']

#--- 결과 확인 (생성된 컬럼들을 중심으로 출력) ---
final_df = merged_df[['행정구역_수정', '결혼이민자 비율', '장애인구 비율', '고령인구 비율', '기초생활수급자 비율', '점포 당 인구수', 'trafic_infra']]
final_df

,행정구역_수정,결혼이민자 비율,장애인구 비율,고령인구 비율,기초생활수급자 비율,점포 당 인구수,trafic_infra
0,강원도 강릉시,0.002250,0.062619,0.237448,0.061075,13361.625000,-2.305463e-08
1,강원도 고성군,0.002553,0.080220,0.302326,0.054619,13905.500000,-1.511527e-09
2,강원도 동해시,0.002668,0.077208,0.238547,0.065958,10777.750000,-3.860876e-08
3,강원도 삼척시,0.002534,0.084007,0.273945,0.064272,12708.400000,-1.110651e-07
4,강원도 속초시,0.002708,0.060623,0.226331,0.070326,10062.125000,-5.004629e-07
...,...,...,...,...,...,...,...
245,충청북도 청주시 상당구,0.002471,0.054155,0.175760,0.050180,12340.687500,-2.164370e-06
246,충청북도 청주시 서원구,0.002295,0.051096,0.175733,0.052632,16268.500000,-5.354256e-06
247,충청북도 청주시 청원구,0.003067,0.045320,0.147587,0.035544,10905.055556,-3.738288e-06
248,충청북도 청주시 흥덕구,0.002953,0.039034,0.128546,0.031475,10422.000000,-6.464511e-06


In [217]:
sigungu = pd.read_csv('data/시군구코드.csv', encoding='utf-8')
sido = pd.read_csv('data/시도코드.csv', encoding = 'cp949', header = None, names = ['시도', '시도코드'])
sigungu['시도코드'] = sigungu['SIGUNGU_CD'].astype(str).str[:2].astype(int)
sigungu = pd.merge(sigungu, sido)
sigungu['행정구역_수정'] = sigungu['시도'] + " " + sigungu['SIGUNGU_NM']
sigungu.drop(columns=['BASE_DATE', 'SIGUNGU_NM', '시도코드', '시도'], inplace = True)
final_df = pd.merge(final_df, sigungu, on = '행정구역_수정')
final_df

,행정구역_수정,결혼이민자 비율,장애인구 비율,고령인구 비율,기초생활수급자 비율,점포 당 인구수,trafic_infra,SIGUNGU_CD
0,강원도 강릉시,0.002250,0.062619,0.237448,0.061075,13361.625000,-2.305463e-08,32030
1,강원도 고성군,0.002553,0.080220,0.302326,0.054619,13905.500000,-1.511527e-09,32600
2,강원도 동해시,0.002668,0.077208,0.238547,0.065958,10777.750000,-3.860876e-08,32040
3,강원도 삼척시,0.002534,0.084007,0.273945,0.064272,12708.400000,-1.110651e-07,32070
4,강원도 속초시,0.002708,0.060623,0.226331,0.070326,10062.125000,-5.004629e-07,32060
...,...,...,...,...,...,...,...,...
245,충청북도 청주시 상당구,0.002471,0.054155,0.175760,0.050180,12340.687500,-2.164370e-06,33041
246,충청북도 청주시 서원구,0.002295,0.051096,0.175733,0.052632,16268.500000,-5.354256e-06,33042
247,충청북도 청주시 청원구,0.003067,0.045320,0.147587,0.035544,10905.055556,-3.738288e-06,33044
248,충청북도 청주시 흥덕구,0.002953,0.039034,0.128546,0.031475,10422.000000,-6.464511e-06,33043


In [218]:
final_df.isnull().sum()

행정구역_수정         0
결혼이민자 비율        0
장애인구 비율         0
고령인구 비율         0
기초생활수급자 비율      0
점포 당 인구수        0
trafic_infra    0
SIGUNGU_CD      0
dtype: int64

In [219]:
col1=final_df.columns[:1].to_list()
col2=final_df.columns[1:].to_list()
new_col=col2+col1
final_df=final_df[new_col]
final_df.head()

,결혼이민자 비율,장애인구 비율,고령인구 비율,기초생활수급자 비율,점포 당 인구수,trafic_infra,SIGUNGU_CD,행정구역_수정
0,0.002250,0.062619,0.237448,0.061075,13361.625,-2.305463e-08,32030,강원도 강릉시
1,0.002553,0.080220,0.302326,0.054619,13905.500,-1.511527e-09,32600,강원도 고성군
2,0.002668,0.077208,0.238547,0.065958,10777.750,-3.860876e-08,32040,강원도 동해시
3,0.002534,0.084007,0.273945,0.064272,12708.400,-1.110651e-07,32070,강원도 삼척시
4,0.002708,0.060623,0.226331,0.070326,10062.125,-5.004629e-07,32060,강원도 속초시


In [220]:
final_df.to_csv('data/final_df.csv', index=False)